# Load Iris Dataset

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import joblib
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

def display_scores(scores):
    print("Scores:", scores)
    print("Means:", scores.mean())
    print("Standard Deviation:", scores.std())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_scores = cross_val_score(rf_clf, X_train, y_train, scoring = 'accuracy', cv = 3)
display_scores(rf_scores)
joblib.dump(rf_clf, 'rf_clf.joblib')

Scores: [0.975 0.9   0.95 ]
Means: 0.9416666666666668
Standard Deviation: 0.03118047822311616


['rf_clf.joblib']

In [66]:
%%writefile flask_api.py

import joblib
import numpy as np
from flask import Flask, request
from flask import Response
import json

# model = None
app = Flask(__name__)

@app.route('/predict')
def predict_endpoint():
    test_data = request.get_json()
    test_data = np.array(test_data)
    predictions = model.predict(test_data)
    response = json.dumps(predictions.tolist())
    return Response(response, status=200, mimetype="application/json")

if __name__ == '__main__':
    model = joblib.load('rf_clf.joblib')
    app.run(host='0.0.0.0', port=80)

Overwriting flask_api.py


In [74]:
from sklearn.metrics import accuracy_score
import requests
import json

url = 'http://localhost/predict'
headers = {
    'Content-type': "application/json"
}

data = json.dumps(X_test.tolist())
response = requests.get(url, headers=headers, data=data)
predictions = np.array(json.loads(response.text))

print(f'Accuracy Score: {accuracy_score(predictions, y_test)}')

Accuracy Score: 1.0
